# Time Series predictions using LSTM

This time we will continue to try different LSTM architectures.
But aside from that we will try to make our process more realistic.
For "the current day" we will only include opening price
as if we were making predictions in the begining of the trading day.

In [1]:
import pandas as pd
from pandas_datareader import data, wb
from datetime import datetime
import numpy as np
import graphviz

import sys
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import bokeh.models
import bokeh.plotting as bk
bk.output_notebook()

Loading BokehJS ...

In [2]:
if not os.path.exists('GOOG.csv'):
    df = data.DataReader('GOOG', 'yahoo', datetime(2010, 1, 1), datetime(2017, 11, 1))
    df.to_csv('GOOG.csv')
else:
    df = pd.read_csv('GOOG.csv')

In [3]:
WINDOW_IN = 7
WINDOW_OUT = 7

In [4]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,311.449310,312.721039,310.103088,311.349976,311.349976,3937800
1,2010-01-05,311.563568,311.891449,308.761810,309.978882,309.978882,6048500
2,2010-01-06,310.907837,310.907837,301.220856,302.164703,302.164703,8009000
3,2010-01-07,302.731018,303.029083,294.410156,295.130463,295.130463,12912000
4,2010-01-08,294.087250,299.675903,292.651581,299.064880,299.064880,9509900


In [5]:
def build_corpus(df, wnd_in=WINDOW_IN, wnd_out=WINDOW_OUT):
    df = df.drop('Date', axis=1)
    
    X, X_columns = [], []
    
    X += [df.Open.shift(k) for k in range(wnd_in + 1)]
    X_columns += ['Open_%s_ago' % k for k in range(wnd_in + 1)]
    X += [df.Low.shift(1+k) for k in range(wnd_in)]
    X_columns += ['Low_%s_ago' % (1+k) for k in range(wnd_in)]
    X += [df.High.shift(1+k) for k in range(wnd_in)]
    X_columns += ['High_%s_ago' % (1+k) for k in range(wnd_in)]
    
    X = pd.concat(X, axis=1)
    X.columns = X_columns
    
    X.loc[:, X_columns] = X.loc[:, X_columns].sub(X.Open_0_ago, axis='rows')
    X.Open_0_ago = df.Open.diff()
    X = X.fillna(0)
    # Let's try without .diff()
    
    y = pd.concat([df.Close.shift(k) for k in range(wnd_out)], axis=1)
    y = y.sub(df.Open, axis='rows')
    y = y.fillna(0)
    y.columns = ['Close_%s' % (k) for k in range(wnd_out)]
    return X, y

In [6]:
X, y = build_corpus(df)

In [7]:
X.head()

,Open_0_ago,Open_1_ago,Open_2_ago,Open_3_ago,Open_4_ago,Open_5_ago,Open_6_ago,Open_7_ago,Low_1_ago,Low_2_ago,...,Low_5_ago,Low_6_ago,Low_7_ago,High_1_ago,High_2_ago,High_3_ago,High_4_ago,High_5_ago,High_6_ago,High_7_ago
0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,0.114258,-0.114258,0.000000,0.000000,0.00000,0.0,0.0,0.0,-1.460480,0.000000,...,0.0,0.0,0.0,1.157471,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,-0.655731,0.655731,0.541473,0.000000,0.00000,0.0,0.0,0.0,-2.146027,-0.804749,...,0.0,0.0,0.0,0.983612,1.813202,0.000000,0.000000,0.0,0.0,0.0
3,-8.176819,8.176819,8.832550,8.718292,0.00000,0.0,0.0,0.0,-1.510162,6.030792,...,0.0,0.0,0.0,8.176819,9.160431,9.990021,0.000000,0.0,0.0,0.0
4,-8.643768,8.643768,16.820587,17.476318,17.36206,0.0,0.0,0.0,0.322906,7.133606,...,0.0,0.0,0.0,8.941833,16.820587,17.804199,18.633789,0.0,0.0,0.0


In [8]:
y.head()

,Close_0,Close_1,Close_2,Close_3,Close_4,Close_5,Close_6
0,-0.099334,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,-1.584686,-0.213592,0.000000,0.000000,0.000000,0.0,0.0
2,-8.743134,-0.928955,0.442139,0.000000,0.000000,0.0,0.0
3,-7.600555,-0.566315,7.247864,8.618958,0.000000,0.0,0.0
4,4.977630,1.043213,8.077453,15.891632,17.262726,0.0,0.0


In [9]:
class RNN3(nn.Module):
    
    
    def __init__(self, n_in, n_mem, n_out):
        super(RNN3, self).__init__()
        self.n_in, self.n_mem, self.n_out = n_in, n_mem, n_out
        self.x2f = nn.Linear(in_features=n_in, out_features=n_mem)
        self.m2f = nn.Linear(in_features=n_mem, out_features=n_mem)
        self.x2invent = nn.Linear(in_features=n_in, out_features=n_mem)
        self.m2invent = nn.Linear(in_features=n_mem, out_features=n_mem)
        self.x2save = nn.Linear(in_features=n_in, out_features=n_mem)
        self.m2save = nn.Linear(in_features=n_mem, out_features=n_mem)
        self.x2pre_o = nn.Linear(in_features=n_in, out_features=n_mem)
        self.pre2o = nn.Linear(in_features=n_mem, out_features=n_out)
        
    def init_weights(self, var=.1):
        for tr in [
                self.x2f,
                self.m2f,
                self.x2invent,
                self.m2invent,
                self.x2save,
                self.m2save,
                self.x2pre_o,
                self.pre2o,
                ]:
            tr.weight.data.uniform_(-var, var)
    
    def forward(self, x, mem):
        xmem = torch.cat((x, mem))
        # chances are, we'll remove too much...
        # but let us try
        mem = mem * F.sigmoid(self.x2f(x)) * F.sigmoid(self.m2f(mem))
        new_mem = F.tanh(self.x2invent(x)) * F.tanh(self.m2invent(mem))
        mem_mask = F.sigmoid(self.x2save(x)) * F.sigmoid(self.m2save(mem))
        mem = mem + new_mem*mem_mask
        pre_o = self.x2pre_o(x)
        o = self.pre2o(F.tanh(mem) * pre_o)
        return (o, mem)
    
    def init_mem(self):
        return Variable(torch.zeros(self.n_mem))

In [10]:
def objective(y_pred, y, alpha=.5):
    n = sum(y.data.shape)
    
    sgn = y_pred*y
    fee = y_pred - y
    fee[sgn.data > 0] = 0
    # abs() grad is bugged in pytorch '0.2.0_4'
    fee = torch.abs(fee)
    fee = fee.sum()
    
    mse = (y_pred - y)**2
    mse = mse.sum()
    return fee + alpha*mse
criterion = objective

In [11]:
def loss(model, X, y,
         objective,
         lr=None,
         max_norm=128,
        ):
    mem = model.init_mem()
    mean_L = 0
    X = torch.Tensor(X.as_matrix())
    Y = torch.Tensor(y.as_matrix())
    for i in range(X.shape[0]):
        model.zero_grad()
        x = Variable(X[i, :])
        y = Variable(Y[i, :])
        y_pred, mem = model(x, mem)
        
        L = objective(y, y_pred)
        mean_L += L.data
        
        if lr is not None:
            L.backward(retain_graph=1)
            torch.nn.utils.clip_grad_norm(model.parameters(), max_norm)
            for p in model.parameters():
                p.data.sub_(lr * p.grad.data)
    mean_L = mean_L/X.shape[0]
    return mean_L

In [12]:
def train(model, X, y, epochs=4, base_lr=.02):
    for epoch in range(1, epochs+1):
        print('Epoch #%s; Loss: %s' % (epoch, loss(model, X, y, objective, lr=base_lr/epoch)))

In [13]:
def predictions(model, X):
    mem = model.init_mem()
    pred = torch.zeros(X.shape[0], model.n_out)
    index = X.index
    X = torch.Tensor(X.as_matrix())
    for i in range(X.shape[0]):
        x = Variable(X[i, :])
        y, mem = model(x, mem)
        pred[i, :] = y.data
    pred = pd.DataFrame(pred.numpy())
    pred.columns = ['Close_%s_next' % (1+k) for k in range(model.n_out)]
    pred.index = index
    return pred

In [22]:
def evaluate_model(model, X, y, base_price=0, title='Model evaluation'):
    pred = predictions(model, X)
    p = bk.figure(
        plot_width=800, plot_height=600,
        title=title,
        active_scroll='wheel_zoom')
    truth = base_price + X.iloc[:, 0].cumsum() + y.iloc[:, 0]
    pred = base_price + X.iloc[:, 0].cumsum() + pred.iloc[:, 0]
    # We'll plot this one to make sure we didn't mess up with diff()'s
    # and base prices.
    # The green line should coincide with the red one.
    p.line(X.index, df.Close[X.index], line_color='green', legend='Ground truth')
    p.line(X.index, truth, line_color='red', legend='Reconstructed original')
    p.line(X.index, pred, line_color='gray', legend='Prediction')
    bk.show(p)

In [15]:
rnn3 = RNN3(n_in=X.shape[1], n_mem=48, n_out=y.shape[1])

In [16]:
n_train = X.shape[0]*2//3
# Skipping first WINDOW_IN rows as the contain NA's
Xtr, ytr = X.iloc[WINDOW_IN:n_train, :], y.iloc[WINDOW_IN:n_train, :]
Xte, yte = X.iloc[n_train:, :], y.iloc[n_train:, :]

In [34]:
train(rnn3, Xtr, ytr, epochs=5, base_lr=.04)

Epoch #1; Loss: 
 11090.9941
[torch.FloatTensor of size 1]

Epoch #2; Loss: 
 1154.3051
[torch.FloatTensor of size 1]

Epoch #3; Loss: 
 462.1208
[torch.FloatTensor of size 1]

Epoch #4; Loss: 
 285.7956
[torch.FloatTensor of size 1]

Epoch #5; Loss: 
 223.5789
[torch.FloatTensor of size 1]



In [39]:
train(rnn3, Xtr, ytr, epochs=8, base_lr=.01)

Epoch #1; Loss: 
 233.1781
[torch.FloatTensor of size 1]

Epoch #2; Loss: 
 132.8372
[torch.FloatTensor of size 1]

Epoch #3; Loss: 
 75.0196
[torch.FloatTensor of size 1]

Epoch #4; Loss: 
 61.0903
[torch.FloatTensor of size 1]

Epoch #5; Loss: 
 56.7817
[torch.FloatTensor of size 1]

Epoch #6; Loss: 
 54.6154
[torch.FloatTensor of size 1]

Epoch #7; Loss: 
 53.3702
[torch.FloatTensor of size 1]

Epoch #8; Loss: 
 52.7222
[torch.FloatTensor of size 1]



In [40]:
evaluate_model(rnn3, Xtr, ytr, base_price=df.Open[WINDOW_IN-1])

In [41]:
evaluate_model(rnn3, Xte, yte, base_price=df.Open[n_train-1])